In [ ]:
# default_exp function_approximation

# Function Approximation

> function approximation playground

`Go to Runtime -> Change runtime type and make sure Hardward accelerator is set to GPU`

In [ ]:
#export
from nbdev.showdoc import *
import jsmltools
from fastcore.test import *
import numpy as np
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.layers.experimental.preprocessing import Rescaling


test_eq(tf.__version__, "2.2.0")

ModuleNotFoundError: No module named 'nbdev'

### Data Setup

In [ ]:
#export

class DataGenerator(object):
    '''generates a multi-calss regression/classification data based on MNIST
    
        x = mnist data flattened
        y = regression/classification targets based on self.initialize()
    
    '''
    def __init__(self, additional_y=0, seed=1123):
        self.additional_y = additional_y
        self.seed = np.random.seed(seed=seed)
        
        # needed to store additional generation functions
        self._func_map = {}
        self._initialize()
        
        assert list(filter(lambda x: x is None, [
            self.mnist_train_x, 
            self.mnist_train_y, 
            self.mnist_test_x, 
            self.mnist_test_y
        ])) == []
        
        self.train_n = len(self.mnist_train_x)
        self.test_n = len(self.mnist_test_x)
       
    def _initialize(self):
        '''prepare functions to approximate '''
        epsilon = 0.000123
        C = 102 # emperical value from analyzing MNIST data
        
        # classification (0, 1) target
        self._func_map[0] = lambda x: int(np.random.random()<0.9) if 2*(x[10]**3)-2*x[3]+15 > 8.9 else 0
        
        # unbounded regression target
        self._func_map[1] = lambda x: np.log(np.sum(x, axis=0)+epsilon)
        
        # bounded (0, 1) regression target
        self._func_map[2] = lambda x: np.mean(x, axis=0)/C
        
        # regular MNIST train/test data
        train, test = tf.keras.datasets.mnist.load_data()
        self.mnist_train_x, self.mnist_train_y = train[0], train[1]
        self.mnist_test_x, self.mnist_test_y = test[0], test[1]
                
    def __call__(self, train_n):
        '''data generation '''
        assert train_n <= self.train_n
        train_x, train_y = [], []
        test_x, test_y = [], []
        
        #TODO (jovsa): consolidate the two loop
        for i in range(train_n):
            x = self.mnist_train_x[i].flatten().reshape(-1, 1)
            y = [self.mnist_train_y[i]]
            for f in range(self.additional_y):
                y.append(self._func_map[f](x))
            train_x.append(x)
            train_y.append(y)
        
        # prepating test data
        for i in range(self.test_n):
            x = self.mnist_test_x[i].flatten().reshape(-1, 1)
            y = [self.mnist_test_y[i]]
            for f in range(self.additional_y):
                y.append(self._func_map[f](x))
            test_x.append(x)
            test_y.append(y)
        
        return train_x, train_y, test_x, test_y

Generating dataset for experimentation

In [ ]:

# data hyperparms
additional_y, train_n = 0, 100
data_generator = DataGenerator(additional_y)
train_x, train_y, test_x, test_y = data_generator(train_n)

# tests
test_eq(len(train_y), train_n)
test_eq(len(train_y[0]), additional_y+1)
test_eq(len(test_y), 10000) # based on MNIST

## Experimenation

Model Based
* Using High Level APIs
    * tf.keras functional 
    * tf.keras sklearn wrapper
* Custom layers
* Custom models
* Custom metrics
* Custom loss
* Custom activation functions
* Custom initializers, regularizers and constants
* Custom training loops
* Custom gradients

Data Loading
 * Tf Example
 * 
Hyperparameter Search


### Using High Level APIs

In [ ]:
# ignore 
# model = tf.keras.Sequential(
#     [
#         keras.layers.Dense(16, input_dim=1, activation='relu'),
#         keras.layers.Dense(8, activation='relu'),
#         keras.layers.Dense(8, activation='relu'),
#         keras.layers.Dense(4, activation='relu'),
#         keras.layers.Dense(1, activation='relu'),
#     ]
# )

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(x=train_x, y=train_y, epochs=200, batch_size=20, verbose=20, validation_split=0.1)

## References
* [Keras for Engineers](https://colab.research.google.com/drive/1lWUGZarlbORaHYUZlF9muCgpPl8pEvve)